In [10]:
!pip3.9 install requests pandas scikit-learn matplotlib python-dotenv

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python3.9 -m pip install --upgrade pip


In [14]:
import requests
from datetime import timedelta, date, datetime as dt
import pandas as pd
from io import StringIO
import numpy as np
from dotenv import load_dotenv
import os

load_dotenv()  

True

# Data Collection

In [4]:
year_data = pd.DataFrame()

In [ ]:
year = 2021


In [15]:
URL="https://www.rescuetime.com/anapi/data"
params = {
"key": os.getenv("RESCUETIME_KEY"),
"perspective": "interval",
"interval": "hour",
"format": "csv",
}

def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days) + 1):
        yield start_date + timedelta(n)


def generate_data(year): 
    year_data =  pd.DataFrame()

    start_date = date(year, 1, 1)
    end_date = date(year, 12, 31)
    
    for single_date in daterange(start_date, end_date):
        params["restrict_begin"] = single_date.strftime("%Y-%m-%d")
        params["restrict_end"] = single_date.strftime("%Y-%m-%d")
        response = requests.get(URL, params=params)
        if response.status_code == 200:
            # Create a StringIO object from the response text and read it into a DataFrame
            data = pd.read_csv(StringIO(response.text))
            # Append the data to the main DataFrame
            year_data = pd.concat([year_data,data], ignore_index=True)
            print(f"Added {single_date.strftime('%Y-%m-%d')}")
        else:
            print(f"Request for {single_date.strftime('%Y-%m-%d')} failed with status code {response.status_code}")
    year_data.to_csv(f'{year}.csv', index=False)
    return year_data




In [16]:
years = [2020,2021,2022]
for year in years:
    generate_data(year)

Added 2020-01-01
Added 2020-01-02
Added 2020-01-03
Added 2020-01-04
Added 2020-01-05
Added 2020-01-06
Added 2020-01-07
Added 2020-01-08
Added 2020-01-09
Added 2020-01-10
Added 2020-01-11
Added 2020-01-12


KeyboardInterrupt: 

In [ ]:
year_data.to_csv(f'{year}.csv', index=False)